In [18]:
import pulsar

In [19]:
client = pulsar.Client('pulsar://localhost:6650')

2022-05-30 14:19:35.393 ERROR [139767119509248] ClientConnection:515 | [127.0.0.1:54284 -> 127.0.0.1:6650] Read failed: Operation canceled
2022-05-30 14:19:35.394 INFO  [139767783630656] ClientConnection:1428 | [127.0.0.1:54284 -> 127.0.0.1:6650] Connection closed
2022-05-30 14:19:35.394 INFO  [139767783630656] ClientConnection:245 | [127.0.0.1:54284 -> 127.0.0.1:6650] Destroyed connection


In [20]:
producer = client.create_producer('request')

2022-05-30 14:19:50.817 INFO  [139767783630656] ConnectionPool:85 | Created connection for pulsar://localhost:6650
2022-05-30 14:19:50.819 INFO  [139767119509248] ClientConnection:356 | [127.0.0.1:54298 -> 127.0.0.1:6650] Connected to broker
2022-05-30 14:19:50.827 INFO  [139767119509248] HandlerBase:54 | [persistent://public/default/request, ] Getting connection from pool
2022-05-30 14:19:50.834 INFO  [139767119509248] ProducerImpl:170 | [persistent://public/default/request, ] Created producer on broker [127.0.0.1:54298 -> 127.0.0.1:6650] 
2022-05-30 14:19:50.834 INFO  [139767783630656] ProducerImpl:527 | Producer - [persistent://public/default/request, standalone-0-2] , [batching  = off]


In [21]:
total_data = 10000

In [22]:
test_stack = []

# Divides the request into batches and sends it to the analysis nodes
def makeRequest(data_size, batch_size=264):
    temp_total = data_size
    start_index = 0
    if not (0 < data_size < 365000):
        raise ValueError("Data Size not available!")
    while data_size > 0:
        if data_size - batch_size < 0:
            batch_size = data_size
        next_index = start_index
        next_index += batch_size
        producer.send((f"{start_index} - {next_index}").encode('utf-8'))
        #test_stack.append(f"{start_index} - {next_index}")
        start_index = next_index + 1
        data_size -= batch_size + 1

In [24]:
makeRequest(20000)

In [25]:
client.close()

2022-05-30 14:22:24.592 INFO  [139767783630656] ClientImpl:483 | Closing Pulsar client
2022-05-30 14:22:24.592 INFO  [139767783630656] ProducerImpl:546 | [persistent://public/default/request, standalone-0-3] Closing producer for topic persistent://public/default/request
2022-05-30 14:22:24.601 INFO  [139767119509248] ProducerImpl:589 | [persistent://public/default/request, standalone-0-3] Closed producer
2022-05-30 14:22:24.601 INFO  [139767119509248] ClientConnection:1428 | [127.0.0.1:54298 -> 127.0.0.1:6650] Connection closed
